# Training thêm data cho sentence bert.
Bài toán sử dụng là dùng text, price tăng hay giảm của gía cổ phiếu ngày hôm đó, từ đó mapping được độ tương quan giữa đoạn text và các cụm từ "giá chứng khoán tăng"
- label > 0.5 => Bài viết có xu hướng thiên về giá chứng khoán tăng.
- label < 0.5 => Bài viết có xu hướng thiên về giá chứng khoán giảm. 

VD:
    text| percentage_increase | text_compare
    "Fed tiếp tục chính sách nâng lãi suất" | 0.3 | "giá chứng khoán tăng"
    "Nhật nới lỏng chính sách tiền tệ" | 0.7 | "giá chứng khoán tăng"


Dùng code từ file:
https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb



In [1]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch


/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_47536/3777221515.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import pandas as pd
from regress_adapter import get_data, get_data_tendency

data = get_data_tendency('TPB', '2022-01-01', '2023-09-29')
print(data.head())


/Users/ngocp/Documents/projects/pyml/botapp/algorithm/regress_adapter.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                                   text  percentage_increase  \
3774  Licogi 14 (L14) “bơm” thêm 214 tỷ đồng đầu tư ...            -1.770231   
3773  Dạo quanh miền duyên hải Việt Nam với các đại ...            -1.770231   
3772  VNDirect: Thị trường đã tạo đáy thành công, đị...             0.000000   
3771  Bất ngờ doanh nghiệp huy động vốn ngàn tỉ qua ...             0.000000   
3770  Giá Bitcoin giảm gần 40% so với đỉnh cao\n\n\n...             0.000000   

          date_y  
3774  2022-01-05  
3773  2022-01-05  
3772  2022-01-06  
3771  2022-01-06  
3770  2022-01-06  


In [4]:
from nis import cat
import re

def exclude_floats(sentence):
    # Regular expression to match float values
    float_pattern = r"\d+\.\d+"
    try:
        # Use re.sub to replace matched float values with an empty string
        result = re.sub(float_pattern, '', sentence)
    except Exception as e:
        print("Exception and wrong sentence:", e, sentence)
        result = ''
    return result

# Example usage:
# sentence = "The price of the product is $19.99, and it weighs 2.5 kg."
# cleaned_sentence = exclude_floats(sentence)
# print('cleaned_sentence', cleaned_sentence)

# data['text_filter_float'] = data['text'].apply(exclude_floats)
# print(data['text_filter_float'].head())

In [5]:
from math import nan


mask = data['text'] != nan
data_t = data[mask]
print(data_t.head())

                                                   text  percentage_increase  \
3774  Licogi 14 (L14) “bơm” thêm 214 tỷ đồng đầu tư ...            -1.770231   
3773  Dạo quanh miền duyên hải Việt Nam với các đại ...            -1.770231   
3772  VNDirect: Thị trường đã tạo đáy thành công, đị...             0.000000   
3771  Bất ngờ doanh nghiệp huy động vốn ngàn tỉ qua ...             0.000000   
3770  Giá Bitcoin giảm gần 40% so với đỉnh cao\n\n\n...             0.000000   

          date_y  
3774  2022-01-05  
3773  2022-01-05  
3772  2022-01-06  
3771  2022-01-06  
3770  2022-01-06  


In [6]:
t = data_t['percentage_increase'].values.reshape(-1, 1)
print('data reshape:', t)
# data_t['movement'] = data_t['percentage_increase'].apply(lambda x: 'chứng khoán giảm' if x < 0 else 'chứng khoán tăng')
# print(data_t['movement'].head())

from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape the "price" column because MinMaxScaler expects a 2D array
data_t['scale_tendency'] = scaler.fit_transform(data_t['percentage_increase'].values.reshape(-1, 1))
print('scale_tendency', data_t['scale_tendency'].head())

data reshape: [[-1.77023121]
 [-1.77023121]
 [ 0.        ]
 ...
 [ 1.40449438]
 [ 1.40449438]
 [ 1.40449438]]
scale_tendency 3774    0.373900
3773    0.373900
3772    0.500949
3771    0.500949
3770    0.500949
Name: scale_tendency, dtype: float64


In [7]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses


def is_float(value):
  """Returns True if the value is a float, False otherwise."""
  return isinstance(value, float)

inputExpTrain = []
for index, row in data_t.iterrows():
    if is_float(row['text']):
        continue

    if row['text'].strip() == '':
        continue
    inputExp = InputExample(texts=[row['text'], 'chứng khoán tăng'], label=row['scale_tendency'])
    inputExpTrain.append(inputExp)


In [8]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

# Load the pre-trained model
model = SentenceTransformer('nli-distilroberta-base-v2')

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(inputExpTrain, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/312 [00:00<?, ?it/s]

In [9]:
model.save('./data/model_tendency')

In [10]:
from sentence_transformers import util

sent_test = [
    '1,4 tỷ dân Trung Quốc cũng không ở hết tất cả nhà bỏ hoang',
    'Quan chức Fed: Sẽ còn ít nhất 1 đợt nâng lãi suất',
    'ASP làm gì để cổ phiếu ra khỏi diện cảnh báo',
    'Cổ phiếu chứng khoán, bất động sản “tan nát” dù xuất hiện dòng tiền bắt đáy',
    'Phiên giảm mạnh của chứng khoán và bất động diễn ra sau khi có thông tin NHNN bắt đầu rút lại thanh khoản trong nền kinh tế trong ngày 21/09. Ngoài ra, hai nhóm cổ phiếu mang nặng tính đầu cơ này đã leo dốc rất mạnh trong thời gian qua và là trụ cột chính cho đà tăng của thị trường',
]
t = model.encode(sent_test)
t1 = model.encode('chứng khoán tăng')
print('tendency', t.shape)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(t, t1)
print(cos_scores)

tendency (5, 768)
tensor([[0.5253],
        [0.5418],
        [0.5034],
        [0.4833],
        [0.4510]])
